# limpiesa y lectura de datos


In [ ]:
from pathlib import Path
#import pandas as pd
import numpy as np
import re
import polars as pd

DATA_DIR = Path("csv").resolve()
assert DATA_DIR.exists(), f"No existe {DATA_DIR}"

# Detectar archivos
asistencia_files = sorted(DATA_DIR.glob("Asistencia_Anual_*.csv"))
rend_files = sorted(DATA_DIR.glob("Rendimiento_*.csv"))

print("Rendimiento:", [f.name for f in rend_files])

# Helper robusto de lectura
def read_csv_smart(path: Path, nrows=None):
    try:
        return pd.read_csv(path, nrows=nrows)
    except UnicodeDecodeError:
        return pd.read_csv(path, nrows=nrows, encoding="latin-1")

Rendimiento: ['Rendimiento_2022_2023.csv', 'Rendimiento_2023_2024.csv', 'Rendimiento_2024_2025.csv']


In [ ]:
def read_csv_smart(path: Path, nrows=None):
    """
    Lee CSV detectando separador y encoding. Omite líneas defectuosas.
    """
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        # 1) auto-inferencia de separador (engine=python)
        try:
            return pd.read_csv(path, nrows=nrows, sep=None, engine="python",
                               encoding=enc, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue
        except pd.errors.ParserError:
            pass
        # 2) intentos explícitos de separador común
        for sep in (";", "\t", "|", ","):
            try:
                return pd.read_csv(path, nrows=nrows, sep=sep, engine="python",
                                   encoding=enc, on_bad_lines="skip")
            except Exception:
                continue
    # 3) último recurso
    return pd.read_csv(path, nrows=nrows, sep=";", engine="python",
                       encoding="latin-1", on_bad_lines="skip")

# Inspección rápida de columnas
for f in asistencia_files + rend_files:
    dfh = read_csv_smart(f, nrows=5)
    print(f"\n{f.name} -> {dfh.shape}")
    print(dfh.columns.tolist())
    display(dfh.head())

# Hints para mapear nombres
COLUMN_HINTS = {
    "id": ["id", "id_alumno", "idalumno", "rut", "estudiante", "id_estudiante"],
    "anio": ["anio", "año", "ano", "periodo", "year"],
    "asistencia": ["asistencia", "asistencia_%", "porc_asistencia", "asistencia_pct"],
    "nota": ["promedio", "nota_final", "nota", "gpa"],
    "aprobado": ["aprobado", "estado", "resultado"]
}

def infer_col(df, keys):
    cols = [c for c in df.columns]
    low = {c.lower(): c for c in cols}
    for k in keys:
        for lc, orig in low.items():
            if re.search(rf"\b{k}\b", lc):
                return orig
    return None


Rendimiento_2022_2023.csv -> (5, 37)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,6",98,P,P,0
1,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
4,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2023_2024.csv -> (5, 37)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
1,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,7",94,P,P,0
2,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,7",100,P,P,0
4,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2024_2025.csv -> (5, 38)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'NOMBRE_SLEP', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,2",99,P,P,0
1,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,2",85,R,R,0
3,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,8",98,P,P,0
4,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0


In [17]:

# Asegurar carpeta de salida
OUT_DIR.mkdir(parents=True, exist_ok=True)

for f in rend_files:
    fpath = Path(f)
    df = read_csv_smart(fpath)

    # normalizar nombres de columna (quitar BOM, espacios, pasar a mayúsculas)
    cols = [str(c).strip().lstrip("\ufeff").upper() for c in df.columns]
    df.columns = cols

    # Crear dataframe de salida con las columnas EXACTAS en el orden requerido
    DFOut = pd.DataFrame(index=df.index)
    for col in REQUIRED_REND:
        if col in df.columns:
            DFOut[col] = df[col]
        else:
            DFOut[col] = np.nan

    # Limpiezas por columna
    if "PROM_GRAL" in DFOut.columns:
        s = DFOut["PROM_GRAL"].astype(str).str.replace(",", ".", regex=False)
        s = s.replace({"nan": np.nan, "": np.nan})
        DFOut["PROM_GRAL"] = pd.to_numeric(s, errors="coerce")

    if "ASISTENCIA" in DFOut.columns:
        s = DFOut["ASISTENCIA"].astype(str).str.replace(",", ".", regex=False).str.replace("%", "", regex=False)
        s = s.replace({"nan": np.nan, "": np.nan})
        DFOut["ASISTENCIA"] = pd.to_numeric(s, errors="coerce")

    for c in ("SIT_FIN", "SIT_FIN_R"):
        if c in DFOut.columns:
            s = DFOut[c].astype(str).str.strip()
            s = s.replace({"nan": np.nan, "": np.nan})
            DFOut[c] = s

    if "MRUN" in DFOut.columns:
        # preservar ceros a la izquierda y limpiar espacios
        DFOut["MRUN"] = DFOut["MRUN"].astype(str).str.strip().replace({"nan": np.nan})

    if "EDAD_ALU" in DFOut.columns:
        DFOut["EDAD_ALU"] = pd.to_numeric(DFOut["EDAD_ALU"], errors="coerce").astype("Int64")

    for c in ("NOM_RBD", "GEN_ALU"):
        if c in DFOut.columns:
            DFOut[c] = DFOut[c].astype(str).str.strip().replace({"nan": np.nan})

    # Guardar en el orden requerido
    out_path = OUT_DIR / (fpath.stem + "_clean.csv")
    DFOut.to_csv(out_path, index=False)
    print("Guardado:", out_path.name)


KeyboardInterrupt: 

# temp


In [19]:


def load_rendimiento_clean(data_dir: str = "datasets/csvClear") -> pd.DataFrame:
    """
    Carga todos los Rendimiento_*_clean.csv generados por datasets/leercsv.ipynb
    y normaliza tipos/claves.
    """
    base = Path(data_dir)
    files = sorted(base.glob("Rendimiento_*_clean.csv"))
    if not files:
        raise FileNotFoundError(f"No se encontraron archivos *_clean en {base}. Ejecuta datasets/leercsv.ipynb primero.")

    dfs = []
    for f in files:
        df = pd.read_csv(f)
        # Normalizar columnas
        df.columns = [str(c).strip().upper().lstrip("\ufeff") for c in df.columns]
        # Asegurar columnas esperadas
        for c in ("AGNO","MRUN","PROM_GRAL","ASISTENCIA","SIT_FIN","SIT_FIN_R"):
            if c not in df.columns:
                df[c] = np.nan

        df["AGNO"] = pd.to_numeric(df["AGNO"], errors="coerce").astype("Int64")
        df["MRUN"] = df["MRUN"].astype(str).str.strip()
        df["PROM_GRAL"] = pd.to_numeric(df["PROM_GRAL"], errors="coerce")
        df["ASISTENCIA"] = pd.to_numeric(df["ASISTENCIA"], errors="coerce")

        for c in ("SIT_FIN","SIT_FIN_R"):
            df[c] = df[c].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})

        df["SOURCE_FILE"] = f.name
        dfs.append(df)

    full = pd.concat(dfs, ignore_index=True)

    # Normalizar asistencia a 0-100 y recortar a rango válido
    mask_01 = full["ASISTENCIA"].between(0, 1, inclusive="both")
    full.loc[mask_01, "ASISTENCIA"] = full.loc[mask_01, "ASISTENCIA"] * 100
    full["ASISTENCIA"] = full["ASISTENCIA"].clip(lower=0, upper=100)

    # Filtrar MRUN vacíos y deduplicar por AGNO+MRUN
    full = full[full["MRUN"].notna() & (full["MRUN"].str.len() > 0)].copy()
    full = full.sort_values(["AGNO","MRUN"]).drop_duplicates(["AGNO","MRUN"], keep="last")

    return full
# ...existing code...

In [20]:
def build_label_aprobacion(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea label binario 'label_aprobado' a partir de SIT_FIN_R / SIT_FIN y,
    si falta, por regla PROM_GRAL >= 4.0.
    1 = Aprobado/Promovido, 0 = Reprobado/Retiro/Deserción/etc.
    """
    df = df.copy()
    y = pd.Series(np.nan, index=df.index, dtype="float")

    # Preferir SIT_FIN_R
    s = df["SIT_FIN_R"].fillna("").str.lower()
    aprob = s.str.contains(r"apro|promov") & ~s.str.contains(r"no\s*apro")
    reprob = s.str.contains(r"reprob|repit|retir|deser|elim|baja|aband")
    y.loc[aprob] = 1
    y.loc[reprob] = 0

    # Fallback SIT_FIN
    s2 = df["SIT_FIN"].fillna("").str.lower()
    aprob2 = s2.str.contains(r"apro|promov") & ~s2.str.contains(r"no\s*apro")
    reprob2 = s2.str.contains(r"reprob|repit|retir|deser|elim|baja|aband")
    y.loc[y.isna() & aprob2] = 1
    y.loc[y.isna() & reprob2] = 0

    # Regla por nota
    nota = pd.to_numeric(df["PROM_GRAL"], errors="coerce")
    y.loc[y.isna() & (nota >= 4.0)] = 1
    y.loc[y.isna() & (nota < 4.0)] = 0

    df["label_aprobado"] = y.astype("Int64")
    return df
# ...existing code...

In [21]:
FEATURE_COLS_EDU = ["prom_gral", "asistencia_pct"]

def engineer_features_edu(df: pd.DataFrame) -> pd.DataFrame:
    """
    Features básicas para educación:
    - prom_gral: nota numérica [1,7]
    - asistencia_pct: asistencia en [0,1]
    - agno: año (para split temporal)
    """
    df = df.copy()
    df["prom_gral"] = pd.to_numeric(df["PROM_GRAL"], errors="coerce").clip(lower=1.0, upper=7.0)
    df["asistencia_pct"] = (pd.to_numeric(df["ASISTENCIA"], errors="coerce")/100.0).clip(lower=0.0, upper=1.0)
    df["agno"] = pd.to_numeric(df["AGNO"], errors="coerce").astype("Int64")
    return df